# Generate the training data (wall + climber + pos)
- Wall: 15 different walls
- Climber: 300 climbers divided into casual, skilled, elite
- Pos: current position (hands position and feet position)

In [4]:
import pandas as pd

# Load the files
walls = pd.read_csv("climbing_data.csv")
climbers = pd.read_csv("all_climbers.csv")

In [5]:
climbers = climbers.drop(columns=["Unnamed: 0"])

In [7]:
from ast import literal_eval
walls["points"] = walls["points"].apply(literal_eval)

In [8]:
import sys
sys.path.append("../src")

from dataset_function import sample_grap_points_by_radius
# Generate the Training Data
times = 20
data = {
    "route": [],
    "climber": [],
    "hands": [],
    "feet": []
}

for w in walls["points"]:
    for _, c in climbers.iterrows():
        for n in range(times):
            hands, feet, center = sample_grap_points_by_radius(route=w,
                                                               climber=c,
                                                               max_trials=100)
            data["route"].append(w)
            data["climber"].append({"group": c["group"],
                                    "height": c["height"],
                                    "weight": c["weight"],
                                    "ape_index": c["ape_index"],
                                    "strength": c["strength"],
                                    "flexibility": c["flexibility"],
                                    "leg_len_factor": c["leg_len_factor"]})
            data["hands"].append(hands)
            data["feet"].append(feet)

In [10]:
df = pd.DataFrame(data)

In [ ]:
df.to_csv("reachability_dataset.csv", index=False)

# Generate the training data (wall + climber + wall features + pos) (for Reachability Model with Holds features)

In [ ]:
# TODO: Create the correct training data: wall + climber + hands + feet + holds_features
"""
wall: 一面墙的全部抓点（中心坐标）
climber: 攀岩者的属性
hands: 目前手的抓点坐标
feet: 目前脚的抓点坐标
holds_features: 该墙每一个抓点的属性 e.g. [{'center_x': 1, 'center_y': 2, 'shape_area':...},{},{}]
"""

In [21]:
import pandas as pd
import numpy as np
from itertools import product
from sklearn.neighbors import NearestNeighbors
import torch
from torch_geometric.data import Data
import sys
sys.path.append("../src")
from dataset_function import sample_hand_foot_points
from dataset_function import get_reachability_label_with_shape

In [22]:
walls_with_features = pd.read_csv("climbing_with_shape_features.csv")
climbers = pd.read_csv("all_climbers.csv").drop(columns=["Unnamed: 0"])

In [ ]:
walls_with_features

,filename,center_x,center_y,shape_area,shape_perimeter,shape_aspect_ratio,shape_circularity
0,0000.jpg,898.200000,1115.866667,8463.0,350.548866,0.790698,0.865440
1,0000.jpg,2064.250000,200.500000,35619.5,698.205728,0.836066,0.918186
2,0000.jpg,2383.750000,1556.250000,5830.0,296.728826,1.207317,0.832068
3,0000.jpg,919.846154,1739.923077,2008.0,166.220468,0.827586,0.913281
4,0000.jpg,713.807692,369.730769,15616.0,561.903101,1.321918,0.621522
...,...,...,...,...,...,...,...
1482,1037.jpg,2487.421053,230.526316,5617.5,304.220222,1.120879,0.762740
1483,1037.jpg,2501.238095,44.285714,5185.0,279.747596,1.197531,0.832580
1484,1037.jpg,1799.444444,482.333333,9680.5,358.154237,0.744186,0.948347
1485,1037.jpg,2675.636364,142.909091,42090.0,822.913427,0.820000,0.781053


In [24]:
unique_walls = walls_with_features["filename"].unique()
wall_climber_pairs = [
    (filename, idx, climbers.loc[idx, "group"])
    for filename, idx in product(unique_walls, climbers.index)
]

In [25]:
graph_list = []
times = 20

for filename, group in walls_with_features.groupby("filename"):
    group = group.dropna(subset=["center_x", "center_y"])
    # Filter out the holds of wall that less than 6
    if len(group) < 6:
        continue

    route_points = group.to_dict(orient='records')

    for _, climber in climbers.iterrows():
        for n in range(times):
            hands, feet = sample_hand_foot_points(route_points, climber)
            if hands is None or feet is None:
                continue
        
            node_feats = []
            labels = []

            for p in route_points:
                node_feats.append([
                    p["center_x"],
                    p["center_y"],
                    p.get("shape_area", 0),
                    p.get("shape_perimeter"),
                    p.get("shape_aspect_ratio", 0),
                    p.get("shape_circularity", 0)
                ])
                labels.append(get_reachability_label_with_shape(p, hands, feet, climber))
            
            label_array = np.array(labels)
            zero_ratio = np.mean(label_array == 0)
            if zero_ratio > 0.95:
                continue

            x = torch.tensor(node_feats, dtype=torch.float)
            y = torch.tensor(labels, dtype=torch.long)

            # Edge construction via KNN
            pos = np.array([[p["center_x"], p["center_y"]] for p in route_points])
            nbrs = NearestNeighbors(n_neighbors=min(8, len(pos))).fit(pos)
            _, indices = nbrs.kneighbors(pos)

            edge_index = []
            for i, neigh in enumerate(indices):
                for j in neigh:
                    if i != j:
                        edge_index.append([i, j])
            edge_index = torch.tensor(edge_index, dtype=torch.long).T

            climber_feat = torch.tensor([
                climber["height"], climber["ape_index"], climber["weight"],
                climber["flexibility"], climber["leg_len_factor"], climber["strength"]
            ], dtype=torch.float).unsqueeze(0)

            graph = Data(x=x, edge_index=edge_index, y=y, climber=climber_feat)

            graph.hands = torch.tensor(hands, dtype=torch.float)
            graph.feet = torch.tensor(feet, dtype=torch.float)

            graph_list.append(graph)

In [26]:
len(graph_list)

89616

In [28]:
graph_list[0]

Data(x=[76, 6], edge_index=[2, 532], y=[76], climber=[1, 6], hands=[2, 2], feet=[2, 2])

In [29]:
torch.save(graph_list, "graph_list_holds_features.pt")